In [1]:
import duckdb

def peek_parquet(file_path, num_rows=5):
    """
    Quickly examine a parquet file using DuckDB
    
    Args:
        file_path: Path to the parquet file

        
        num_rows: Number of rows to preview (default 5)
    """
    # Connect to DuckDB
    con = duckdb.connect()
    
    # Get schema information
    schema_query = f"""
    SELECT column_name, data_type 
    FROM duckdb_columns 
    WHERE table_name = 'parquet_scan' 
    AND schema_name = 'main'
    """
    con.execute(f"CREATE VIEW parquet_scan AS SELECT * FROM parquet_scan('{file_path}')")
    schema = con.execute(schema_query).fetchall()
    
    print("Schema Information:")
    print("-----------------")
    for col, dtype in schema:
        print(f"{col}: {dtype}")
    print("\nSample Data:")
    print("-----------------")
    
    # Get sample rows
    sample_query = f"SELECT * FROM parquet_scan('{file_path}') LIMIT {num_rows}"
    sample_data = con.execute(sample_query).fetchall()
    
    # Get column names for pretty printing
    columns = [col for col, _ in schema]
    
    # Print sample data
    for row in sample_data:
        for col, val in zip(columns, row):
            print(f"{col}: {val}")
        print("-----------------")
    
    # Get basic statistics
    stats_query = f"""
    SELECT 
        COUNT(*) as row_count,
        COUNT(DISTINCT *) as distinct_rows
    FROM parquet_scan('{file_path}')
    """
    stats = con.execute(stats_query).fetchone()
    
    print("\nFile Statistics:")
    print("-----------------")
    print(f"Total Rows: {stats[0]}")
    print(f"Distinct Rows: {stats[1]}")
    
    # Close connection
    con.close()

# Usage example:
# peek_parquet('path/to/your/file.parquet')

In [ ]:
peek_parquet('../data/urls-meta-23-11-02.parquet', num_rows=10)

In [ ]:
import duckdb

# Initialize DuckDB connection
conn = duckdb.connect()
feeds = "../data/feeds.csv"
# Read the feeds CSV file and normalize URLs in one step
conn.execute(f"""
    CREATE TABLE normalized_feeds AS 
    SELECT 
        column0 as original_url,
        column2 as feed_url,
        CASE 
            WHEN column0 LIKE 'www.%' THEN 'https://' || column0
            ELSE 'https://www.' || column0
        END as normalized_url
    FROM read_csv('{feeds}', header=False);
""")


In [4]:
def filter_urls_by_patterns(parquet_path: str, patterns: list[str], conn) -> list[str]:
    """
    Filter URLs from a parquet file based on specified ending patterns.
    Performs the filtering during the initial read to be memory efficient.
    
    Args:
        parquet_path (str): Path to the parquet file containing URLs
        patterns (list[str]): List of URL endings to filter by (e.g. ['about', 'about-us'])
        conn: DuckDB connection object
    
    Returns:
        list[str]: List of filtered URLs
    """
    # Create pattern string for the SQL query
    pattern_values = ", ".join(f"'{pattern}'" for pattern in patterns)
    
    # Combined query that reads parquet and filters in one step
    query = f"""
    WITH patterns AS (
        SELECT unnest([{pattern_values}]) as pattern
    )
    SELECT column0 as url
    FROM read_parquet('{parquet_path}')
    WHERE EXISTS (
        SELECT 1 
        FROM patterns 
        WHERE column0 LIKE '%/' || pattern
    );
    """
    
    result = conn.execute(query).fetchall()
    return [row[0] for row in result]

# Example usage:
# import duckdb
# conn = duckdb.connect(database=':memory:')
# about_patterns = ['about', 'about-us', 'about-company']
urls_path = "../data/urls-meta-23-11-02.parquet"
filtered_urls = filter_urls_by_patterns(urls_path, ABOUT_PATTERNS, conn)

RuntimeError: Query interrupted

In [3]:
ABOUT_PATTERNS = [
    # English patterns
    "/about",
    "/about-me",
    "/bio",
    "/biography",
    "/me",
    "/who-am-i",
    "/about/me",
    "/hello",
    "/introduction",
    "/personal",
    "/profile",
    "/my-story",
    "/my-journey",
    "/about-the-author",
    "/meet-me",
    # Spanish patterns
    "/sobre-mi",
    "/quien-soy",
    "/acerca-de-mi",
    "/mi-biografia",
    "/mi-historia",
    "/biografia",
    "/presentacion",
    "/perfil",
    "/conoceme",
    "/hola",
    # German patterns
    "/ueber-mich",
    "/über-mich",
    "/ich",
    "/meine-geschichte",
    "/biografie",
    "/das-bin-ich",
    "/steckbrief",
    "/vorstellung",
    "/hallo",
    "/personliches",
    "/persönliches",
    # French patterns
    "/a-propos",
    "/qui-suis-je",
    "/biographie",
    "/ma-bio",
    "/mon-parcours",
    "/me-connaitre",
    "/me-connaître",
    "/presentation",
    "/présentation",
    "/bonjour",
    "/mon-histoire",
]

In [ ]:


# Perform the join with simple pattern matching
conn.execute("""
    CREATE TABLE result AS
SELECT 
    nf.*,
    u.url as matched_url,
    u.column2,
    u.column3
FROM normalized_feeds nf
LEFT JOIN urls_meta u
ON (
    -- Handle cases with www
    REGEXP_REPLACE(u.url, 'https?://(?:www\.)?([^/]+).*', '\1') =
    REGEXP_REPLACE(nf.original_url, 'https?://(?:www\.)?([^/]+).*', '\1')
);CREATE TABLE result AS
    SELECT 
        nf.*,
        u.url as matched_url,
        u.column2,
        u.column3
    FROM normalized_feeds nf
    LEFT JOIN urls_meta u
    ON (u.url LIKE 'https://www.' || REPLACE(nf.original_url, 'www.', '') || '%'
        OR u.url LIKE 'https://' || REPLACE(nf.original_url, 'www.', '') || '%');
""")

# Write the results to a new parquet file
conn.execute("""
    COPY result TO 'merged_results.parquet' (FORMAT PARQUET);
""")

# Print some statistics about the join
conn.execute("""
    SELECT 
        COUNT(*) as total_matches,
        COUNT(DISTINCT original_url) as unique_feeds,
        COUNT(DISTINCT matched_url) as unique_matches
    FROM result WHERE matched_url IS NOT NULL;
""").fetchall()

# Close the connection
conn.close()